# FLUXNET CO₂ Flux Analysis: Spain vs Ireland

## Eddy-Covariance Measurement Behaviour Across Mediterranean and Atlantic Climate Regimes

**Author context:** Currently based near Gibraltar / southern Spain, applying for a Teagasc Research Officer role in Ireland.

**Motivation:** Before attempting process-model validation of carbon fluxes, it is essential to understand the real measurement Behaviour, quality control challenges, and inherent uncertainty in eddy-covariance flux data. This notebook provides an exploratory analysis of FLUXNET2015 observations.

---

### What This Notebook Does
- Loads and harmonizes CO₂ flux data from 6 eddy-covariance sites (3 Spanish, 3 Irish)
- Implements systematic quality control with transparent flagging
- Computes cumulative carbon budgets with uncertainty estimates
- Explores water-stress signals (VPD/soil moisture) as drivers of flux Behaviour
- Visualizes seasonal patterns and cross-site comparisons

### What This Notebook Does NOT Claim
- ❌ No long-term soil organic carbon (SOC) inference
- ❌ No process-model fitting or parameter calibration
- ❌ No gap-filling algorithm development
- ❌ No attribution of flux changes to management practices

*This is honest, exploratory measurement analysis—not ecosystem forecasting.*

---

### Key Definitions

**Flux:** Rate of CO₂ exchange between the land surface and atmosphere, typically measured in μmol CO₂ m⁻² s⁻¹ (instantaneous) or g C m⁻² day⁻¹ (daily).

**Sign convention (ecological):**
- **Negative NEE** → net carbon uptake by the ecosystem (photosynthesis > respiration)
- **Positive NEE** → net carbon release to atmosphere (respiration > photosynthesis)

**Eddy covariance:** A micrometeorological technique that directly measures the turbulent exchange of CO₂ between surfaces and atmosphere using high-frequency (10-20 Hz) measurements of vertical wind velocity and CO₂ concentration. The covariance of these fluctuations gives the flux.

**VPD (Vapor Pressure Deficit):** Difference between saturation and actual water vapor pressure; a key indicator of atmospheric dryness and evaporative demand that strongly affects plant stomatal Behaviour.

---

### Why Spain vs Ireland?

| Aspect | Spain (Mediterranean) | Ireland (Atlantic) |
|--------|----------------------|-------------------|
| Summer rainfall | Scarce (dry season) | Moderate-abundant |
| VPD regime | High summer VPD, stomatal closure | Generally low VPD |
| Growing season | Winter-spring peak | Spring-summer peak |
| Water limitation | Primary constraint | Rarely limiting |

**Hypothesis:** Spanish sites should show stronger "uptake collapse" during high-VPD summer months, while Irish sites maintain more consistent uptake throughout the growing season.


---

## Data Source and Download Instructions

### FLUXNET2015 Dataset

Data for this analysis comes from the [FLUXNET2015 dataset](https://fluxnet.org/data/fluxnet2015-dataset/), a globally-standardized collection of eddy-covariance flux tower measurements.

### Sites Used

| Site ID | Site Name | Country | Lat | Lon | IGBP Class | Notes |
|---------|-----------|---------|-----|-----|------------|-------|
| ES-LgS | Laguna Seca | Spain | 37.0979 | -2.9658 | CSH | Semi-arid shrubland, Almería province |
| ES-LJu | Llano de los Juanes | Spain | 36.9266 | -2.7521 | OSH | Open shrubland, semi-arid climate |
| ES-LMa | Las Majadas del Tietar | Spain | 39.9415 | -5.7734 | SAV | Dehesa savanna, Mediterranean oak |
| IE-Dri | Dripsey | Ireland | 51.9867 | -8.7514 | GRA | Grassland, temperate oceanic climate |
| IE-Ca1 | Carlow Crop | Ireland | 52.8550 | -6.9000 | CRO | Cropland rotation |
| IE-Ca2 | Carlow Grass | Ireland | 52.8600 | -6.9000 | GRA | Managed grassland |

*IGBP Classes: CSH=Closed Shrubland, OSH=Open Shrubland, SAV=Savannas, GRA=Grasslands, CRO=Croplands*

### Download Steps

1. Visit the FLUXNET2015 site page for each site:
   - https://fluxnet.org/doi/FLUXNET2015/ES-LgS
   - https://fluxnet.org/doi/FLUXNET2015/ES-LJu
   - https://fluxnet.org/doi/FLUXNET2015/ES-LMa
   - https://fluxnet.org/doi/FLUXNET2015/IE-Dri
   - https://fluxnet.org/doi/FLUXNET2015/IE-Ca1
   - https://fluxnet.org/doi/FLUXNET2015/IE-Ca2

2. Request access (free for research use; requires registration)

3. Download the FULLSET ZIP archive for each site

4. Place downloaded ZIPs in: `data/raw/FLUXNET2015/`

5. Extract CSVs to: `data/extracted/<SITE_ID>/`

### Expected Folder Structure

```
project_root/
├── fluxnet_spain_vs_ireland.ipynb   # This notebook
├── data/
│   ├── raw/
│   │   └── FLUXNET2015/
│   │       ├── FLX_ES-LgS_FLUXNET2015_FULLSET_*.zip
│   │       ├── FLX_ES-LJu_FLUXNET2015_FULLSET_*.zip
│   │       └── ... (other site ZIPs)
│   └── extracted/
│       ├── ES-LgS/
│       │   └── FLX_ES-LgS_FLUXNET2015_FULLSET_DD_*.csv
│       ├── ES-LJu/
│       │   └── ...
│       └── ... (other sites)
└── outputs/
    ├── figures/
    └── tables/
```

**Note:** This notebook does NOT hardcode any private tokens or credentials.


---

## 1. Setup and Configuration


In [1]:
# ============================================================================
# Imports and Configuration
# ============================================================================

import os
import glob
import warnings
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

# Optional imports with graceful degradation
try:
    import folium
    from folium import plugins
    FOLIUM_AVAILABLE = True
except ImportError:
    FOLIUM_AVAILABLE = False
    print("Note: folium not installed. Map visualization will be skipped.")

try:
    from scipy import stats
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False
    print("Note: scipy not installed. Some statistical tests will be skipped.")

# Suppress common warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 100,
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'axes.edgecolor': '#333333',
    'axes.linewidth': 1.0,
    'grid.alpha': 0.3,
})

# Color palettes for Spain vs Ireland
SPAIN_COLORS = ['#E74C3C', '#E67E22', '#F39C12']  # Warm reds/oranges
IRELAND_COLORS = ['#27AE60', '#2ECC71', '#1ABC9C']  # Cool greens
ALL_SITE_COLORS = {
    'ES-LgS': '#E74C3C', 'ES-LJu': '#E67E22', 'ES-LMa': '#F39C12',
    'IE-Dri': '#27AE60', 'IE-Ca1': '#2ECC71', 'IE-Ca2': '#1ABC9C'
}

# Random seed for reproducibility
np.random.seed(42)

print("✓ All core imports successful")
print(f"  - pandas {pd.__version__}")
print(f"  - numpy {np.__version__}")
print(f"  - matplotlib {plt.matplotlib.__version__}")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# ============================================================================
# Site Metadata Configuration
# ============================================================================

SITE_METADATA = {
    # Spanish sites (Mediterranean)
    'ES-LgS': {
        'name': 'Laguna Seca',
        'country': 'Spain',
        'lat': 37.0979,
        'lon': -2.9658,
        'igbp': 'CSH',  # Closed Shrubland
        'igbp_full': 'Closed Shrubland',
        'climate': 'Mediterranean',
        'notes': 'Semi-arid shrubland in Almería province; subject to severe summer drought.'
    },
    'ES-LJu': {
        'name': 'Llano de los Juanes',
        'country': 'Spain',
        'lat': 36.9266,
        'lon': -2.7521,
        'igbp': 'OSH',  # Open Shrubland
        'igbp_full': 'Open Shrubland',
        'climate': 'Mediterranean',
        'notes': 'Open shrubland in semi-arid zone; sparse vegetation cover.'
    },
    'ES-LMa': {
        'name': 'Las Majadas del Tietar',
        'country': 'Spain',
        'lat': 39.9415,
        'lon': -5.7734,
        'igbp': 'SAV',  # Savannas
        'igbp_full': 'Savannas (Dehesa)',
        'climate': 'Mediterranean',
        'notes': 'Dehesa ecosystem with scattered Quercus ilex; traditional silvopastoral system.'
    },
    # Irish sites (Atlantic)
    'IE-Dri': {
        'name': 'Dripsey',
        'country': 'Ireland',
        'lat': 51.9867,
        'lon': -8.7514,
        'igbp': 'GRA',  # Grasslands
        'igbp_full': 'Grasslands',
        'climate': 'Atlantic',
        'notes': 'Managed grassland in County Cork; humid temperate climate.'
    },
    'IE-Ca1': {
        'name': 'Carlow Crop',
        'country': 'Ireland',
        'lat': 52.8550,
        'lon': -6.9000,
        'igbp': 'CRO',  # Croplands
        'igbp_full': 'Croplands',
        'climate': 'Atlantic',
        'notes': 'Arable cropland in rotation (cereals); Teagasc research farm.'
    },
    'IE-Ca2': {
        'name': 'Carlow Grass',
        'country': 'Ireland',
        'lat': 52.8600,
        'lon': -6.9000,
        'igbp': 'GRA',  # Grasslands
        'igbp_full': 'Grasslands',
        'climate': 'Atlantic',
        'notes': 'Intensively managed grassland; adjacent to IE-Ca1 cropland site.'
    }
}

SPANISH_SITES = ['ES-LgS', 'ES-LJu', 'ES-LMa']
IRISH_SITES = ['IE-Dri', 'IE-Ca1', 'IE-Ca2']
ALL_SITES = SPANISH_SITES + IRISH_SITES

# Path configuration
DATA_RAW = Path('data/raw/FLUXNET2015')
DATA_EXTRACTED = Path('data/extracted')
OUTPUT_FIGURES = Path('outputs/figures')
OUTPUT_TABLES = Path('outputs/tables')

# Ensure output directories exist
OUTPUT_FIGURES.mkdir(parents=True, exist_ok=True)
OUTPUT_TABLES.mkdir(parents=True, exist_ok=True)

print(f"Configured {len(ALL_SITES)} sites:")
print(f"  Spain: {', '.join(SPANISH_SITES)}")
print(f"  Ireland: {', '.join(IRISH_SITES)}")


In [ ]:
# ============================================================================
# File Availability Check
# ============================================================================

def check_data_availability() -> Dict[str, Dict]:
    """
    Check which site data files are available.
    Returns a dict with status for each site.
    """
    status = {}
    
    for site_id in ALL_SITES:
        site_dir = DATA_EXTRACTED / site_id
        status[site_id] = {
            'dir_exists': site_dir.exists(),
            'csv_files': [],
            'daily_file': None,
            'hourly_file': None,
            'halfhourly_file': None,
        }
        
        if site_dir.exists():
            # Look for FLUXNET CSV files
            csv_files = list(site_dir.glob('FLX_*.csv'))
            status[site_id]['csv_files'] = [f.name for f in csv_files]
            
            # Identify temporal resolution files
            for f in csv_files:
                fname = f.name.upper()
                if '_DD_' in fname:  # Daily
                    status[site_id]['daily_file'] = f
                elif '_HH_' in fname:  # Hourly
                    status[site_id]['hourly_file'] = f
                elif '_HR_' in fname:  # Half-hourly
                    status[site_id]['halfhourly_file'] = f
    
    return status

# Run check
data_status = check_data_availability()

# Display results
print("=" * 70)
print("DATA AVAILABILITY CHECK")
print("=" * 70)

found_sites = []
missing_sites = []

for site_id in ALL_SITES:
    s = data_status[site_id]
    meta = SITE_METADATA[site_id]
    
    if s['daily_file'] is not None:
        found_sites.append(site_id)
        status_str = f"✓ FOUND (daily file: {s['daily_file'].name})"
    elif s['hourly_file'] is not None or s['halfhourly_file'] is not None:
        found_sites.append(site_id)
        res = 'hourly' if s['hourly_file'] else 'half-hourly'
        status_str = f"✓ FOUND ({res} - will aggregate to daily)"
    elif s['dir_exists']:
        missing_sites.append(site_id)
        status_str = f"⚠ Directory exists but no FLUXNET CSV found"
    else:
        missing_sites.append(site_id)
        status_str = f"✗ NOT FOUND (directory missing)"
    
    country_flag = "🇪🇸" if site_id.startswith("ES") else "🇮🇪"
    print(f"\n{country_flag} {site_id} ({meta['name']})")
    print(f"   {status_str}")
    if s['csv_files'] and s['daily_file'] is None:
        print(f"   Files found: {s['csv_files'][:3]}...")  # Show first 3

print("\n" + "=" * 70)
print(f"Summary: {len(found_sites)}/{len(ALL_SITES)} sites have usable data")
if missing_sites:
    print(f"Missing: {', '.join(missing_sites)}")
    print("\n⚠ Please download missing data following the instructions above.")
print("=" * 70)


---

## 2. Data Ingestion

### FLUXNET Variable Naming Convention

FLUXNET2015 uses a structured naming convention for variables:

- `NEE_VUT_REF` — Net Ecosystem Exchange, Variable U-star Threshold, Reference (preferred)
- `NEE_VUT_MEAN` — NEE, Variable U-star Threshold, Mean across u* thresholds
- `TA_F` — Air Temperature, gap-filled
- `SW_IN_F` — Incoming Shortwave Radiation, gap-filled  
- `VPD_F` — Vapor Pressure Deficit, gap-filled
- `SWC_F_MDS_1` — Soil Water Content, gap-filled (MDS method), layer 1

**Quality flags:** Variables ending in `_QC` indicate data quality (0=measured, 1=good gap-fill, 2=medium, 3=poor)

### Timestamp Handling

FLUXNET daily files use `TIMESTAMP` in format `YYYYMMDD`. We parse this to proper datetime objects and create derived columns (year, month, day of year).


In [ ]:
# ============================================================================
# Core Data Loading Functions
# ============================================================================

# Variables we want to extract (in order of preference)
TARGET_VARIABLES = {
    'nee': ['NEE_VUT_REF', 'NEE_VUT_MEAN', 'NEE_CUT_REF', 'NEE_CUT_MEAN'],
    'temp': ['TA_F', 'TA_F_MDS', 'TA_ERA'],
    'radiation': ['SW_IN_F', 'SW_IN_F_MDS', 'SW_IN_ERA'],
    'vpd': ['VPD_F', 'VPD_F_MDS', 'VPD_ERA'],
    'swc': ['SWC_F_MDS_1', 'SWC_F_MDS_2', 'SWC_F_MDS_3']  # Soil water, different layers
}

def find_best_variable(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """
    Find the first available variable from a list of candidates.
    Returns column name if found, None otherwise.
    """
    for var in candidates:
        if var in df.columns:
            # Check it has at least some valid data
            valid_count = df[var].notna().sum()
            if valid_count > 0:
                return var
    return None

def parse_fluxnet_timestamp(ts_series: pd.Series) -> pd.DatetimeIndex:
    """
    Parse FLUXNET timestamp format (YYYYMMDD for daily, YYYYMMDDhhmm for sub-daily).
    Handles both integer and string formats.
    """
    # Convert to string if needed
    ts_str = ts_series.astype(str)
    
    # Determine format based on length
    sample_len = len(ts_str.iloc[0])
    
    if sample_len == 8:  # YYYYMMDD (daily)
        return pd.to_datetime(ts_str, format='%Y%m%d')
    elif sample_len == 12:  # YYYYMMDDhhmm (sub-daily)
        return pd.to_datetime(ts_str, format='%Y%m%d%H%M')
    else:
        # Fallback: let pandas infer
        return pd.to_datetime(ts_str)

def load_site_data(site_id: str, data_status: Dict = None) -> Optional[pd.DataFrame]:
    """
    Load FLUXNET data for a single site.
    
    Parameters:
    -----------
    site_id : str
        Site identifier (e.g., 'ES-LgS')
    data_status : dict
        Pre-computed data availability status (optional)
    
    Returns:
    --------
    pd.DataFrame with standardized columns:
        - date: datetime index
        - nee_raw: NEE in original units (g C m-2 day-1 for daily)
        - temp: air temperature (°C)
        - radiation: incoming shortwave (W m-2)
        - vpd: vapor pressure deficit (hPa)
        - swc: soil water content (%)
        - year, month, doy: derived time columns
        - *_qc: quality flags where available
    
    Returns None if data cannot be loaded.
    """
    if data_status is None:
        data_status = check_data_availability()
    
    status = data_status.get(site_id, {})
    
    # Determine which file to use (prefer daily)
    data_file = status.get('daily_file')
    temporal_res = 'daily'
    
    if data_file is None:
        data_file = status.get('hourly_file') or status.get('halfhourly_file')
        temporal_res = 'sub-daily'
    
    if data_file is None:
        print(f"⚠ No data file found for {site_id}")
        return None
    
    print(f"Loading {site_id} from {data_file.name} ({temporal_res})...")
    
    try:
        # Load CSV with FLUXNET conventions
        # -9999 is the standard missing value indicator
        df = pd.read_csv(data_file, na_values=['-9999', -9999, '-9999.0'])
        
        # Parse timestamp
        if 'TIMESTAMP' in df.columns:
            df['date'] = parse_fluxnet_timestamp(df['TIMESTAMP'])
        elif 'TIMESTAMP_START' in df.columns:
            df['date'] = parse_fluxnet_timestamp(df['TIMESTAMP_START'])
        else:
            print(f"  ⚠ No timestamp column found!")
            return None
        
        # If sub-daily, aggregate to daily
        if temporal_res == 'sub-daily':
            df = aggregate_to_daily(df)
        
        # Extract target variables
        result = pd.DataFrame({'date': df['date']})
        result['site_id'] = site_id
        
        # NEE
        nee_col = find_best_variable(df, TARGET_VARIABLES['nee'])
        if nee_col:
            result['nee_raw'] = df[nee_col].values
            result['nee_source'] = nee_col
            # Get QC flag if available
            nee_qc = nee_col.replace('_REF', '_QC').replace('_MEAN', '_QC')
            if nee_qc in df.columns:
                result['nee_qc'] = df[nee_qc].values
        else:
            print(f"  ⚠ No NEE variable found")
            result['nee_raw'] = np.nan
        
        # Temperature
        temp_col = find_best_variable(df, TARGET_VARIABLES['temp'])
        if temp_col:
            result['temp'] = df[temp_col].values
        else:
            result['temp'] = np.nan
        
        # Radiation
        rad_col = find_best_variable(df, TARGET_VARIABLES['radiation'])
        if rad_col:
            result['radiation'] = df[rad_col].values
        else:
            result['radiation'] = np.nan
        
        # VPD (Vapor Pressure Deficit)
        vpd_col = find_best_variable(df, TARGET_VARIABLES['vpd'])
        if vpd_col:
            result['vpd'] = df[vpd_col].values
        else:
            result['vpd'] = np.nan
        
        # SWC (Soil Water Content)
        swc_col = find_best_variable(df, TARGET_VARIABLES['swc'])
        if swc_col:
            result['swc'] = df[swc_col].values
        else:
            result['swc'] = np.nan
        
        # Derived time columns
        result['year'] = result['date'].dt.year
        result['month'] = result['date'].dt.month
        result['doy'] = result['date'].dt.dayofyear
        
        # Set date as index
        result = result.set_index('date')
        
        print(f"  ✓ Loaded {len(result)} days, {result['year'].min()}-{result['year'].max()}")
        
        return result
        
    except Exception as e:
        print(f"  ✗ Error loading {site_id}: {e}")
        return None

def aggregate_to_daily(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate sub-daily (hourly/half-hourly) data to daily resolution.
    Uses appropriate aggregation for each variable type:
    - NEE: sum (g C m-2 day-1)
    - Temperature, VPD: mean
    - Radiation: mean (daily average W m-2)
    """
    df = df.copy()
    df['date_day'] = df['date'].dt.date
    
    # Define aggregation rules
    agg_rules = {}
    
    for col in df.columns:
        if col in ['date', 'date_day', 'TIMESTAMP', 'TIMESTAMP_START', 'TIMESTAMP_END']:
            continue
        
        col_upper = col.upper()
        
        # NEE variables should be summed (after scaling)
        if 'NEE' in col_upper:
            # Note: FLUXNET sub-daily NEE is in μmol CO2 m-2 s-1
            # Need to convert to daily sum (this is simplified)
            agg_rules[col] = 'mean'  # We'll use mean and note this limitation
        # Temperature and VPD: mean
        elif any(x in col_upper for x in ['TA_', 'VPD_', 'RH_']):
            agg_rules[col] = 'mean'
        # Radiation: mean
        elif any(x in col_upper for x in ['SW_', 'LW_', 'PPFD']):
            agg_rules[col] = 'mean'
        # Soil water: mean
        elif 'SWC' in col_upper:
            agg_rules[col] = 'mean'
        # QC flags: take max (worst quality)
        elif '_QC' in col_upper:
            agg_rules[col] = 'max'
        # Default: mean
        else:
            agg_rules[col] = 'mean'
    
    # Aggregate
    daily = df.groupby('date_day').agg(agg_rules).reset_index()
    daily['date'] = pd.to_datetime(daily['date_day'])
    daily = daily.drop(columns=['date_day'])
    
    return daily

print("✓ Data loading functions defined")


In [ ]:
# ============================================================================
# Load All Sites
# ============================================================================

# Dictionary to store all site data
site_data = {}

print("Loading data for all sites...")
print("=" * 70)

for site_id in ALL_SITES:
    df = load_site_data(site_id, data_status)
    if df is not None:
        site_data[site_id] = df

print("\n" + "=" * 70)
print(f"Successfully loaded: {len(site_data)}/{len(ALL_SITES)} sites")

if len(site_data) == 0:
    print("\n⚠ NO DATA LOADED!")
    print("Please download FLUXNET2015 data following the instructions above.")
    print("The remaining cells will demonstrate the analysis structure but")
    print("will use synthetic data for illustration purposes.")
    
    # Create synthetic data for demonstration
    print("\nGenerating synthetic demonstration data...")
    
    for site_id in ALL_SITES:
        np.random.seed(hash(site_id) % 2**31)
        
        # Generate 3 years of daily data
        dates = pd.date_range('2010-01-01', '2012-12-31', freq='D')
        n = len(dates)
        
        # Seasonal pattern
        doy = dates.dayofyear
        seasonal = -3 * np.sin(2 * np.pi * (doy - 100) / 365)  # Peak uptake in spring
        
        # Add site-specific characteristics
        if site_id.startswith('ES'):  # Spanish sites: summer stress
            summer_stress = 2 * np.exp(-((doy - 200) ** 2) / (2 * 30 ** 2))
            nee = seasonal + summer_stress + np.random.normal(0, 1.5, n)
            temp = 15 + 10 * np.sin(2 * np.pi * (doy - 100) / 365) + np.random.normal(0, 2, n)
            vpd = 10 + 15 * np.sin(2 * np.pi * (doy - 100) / 365) + np.random.normal(0, 3, n)
            vpd = np.clip(vpd, 0, None)
        else:  # Irish sites: more consistent
            nee = seasonal + np.random.normal(0, 1.2, n)
            temp = 10 + 5 * np.sin(2 * np.pi * (doy - 100) / 365) + np.random.normal(0, 2, n)
            vpd = 5 + 5 * np.sin(2 * np.pi * (doy - 100) / 365) + np.random.normal(0, 2, n)
            vpd = np.clip(vpd, 0, None)
        
        # Radiation pattern
        radiation = 150 + 150 * np.sin(2 * np.pi * (doy - 80) / 365) + np.random.normal(0, 30, n)
        radiation = np.clip(radiation, 0, None)
        
        # Create DataFrame
        df = pd.DataFrame({
            'site_id': site_id,
            'nee_raw': nee,
            'nee_source': 'SYNTHETIC',
            'temp': temp,
            'radiation': radiation,
            'vpd': vpd,
            'swc': np.nan,  # Not always available
            'year': dates.year,
            'month': dates.month,
            'doy': doy
        }, index=dates)
        
        site_data[site_id] = df
    
    print(f"✓ Generated synthetic data for {len(site_data)} sites")
    print("⚠ Note: Using SYNTHETIC data for demonstration. Replace with real FLUXNET data!")


In [ ]:
# ============================================================================
# Data Summary
# ============================================================================

def summarize_loaded_data(site_data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """Create a summary table of loaded site data."""
    rows = []
    
    for site_id, df in site_data.items():
        meta = SITE_METADATA[site_id]
        
        # Calculate statistics
        nee_valid = df['nee_raw'].notna().sum()
        nee_total = len(df)
        nee_pct = 100 * nee_valid / nee_total
        
        rows.append({
            'Site': site_id,
            'Name': meta['name'],
            'Country': meta['country'],
            'IGBP': meta['igbp'],
            'Start': df.index.min().strftime('%Y-%m-%d'),
            'End': df.index.max().strftime('%Y-%m-%d'),
            'Days': len(df),
            'Years': df['year'].nunique(),
            'NEE Valid (%)': f"{nee_pct:.1f}",
            'NEE Source': df['nee_source'].iloc[0] if 'nee_source' in df.columns else 'N/A'
        })
    
    return pd.DataFrame(rows)

# Create and display summary
summary_df = summarize_loaded_data(site_data)
print("\n📊 DATA SUMMARY")
print("=" * 100)
print(summary_df.to_string(index=False))

# Save summary table
summary_df.to_csv(OUTPUT_TABLES / 'site_data_summary.csv', index=False)
print(f"\n✓ Summary saved to {OUTPUT_TABLES / 'site_data_summary.csv'}")


---

## 3. Quality Control (QC) Module

### QC Philosophy

Quality control in flux data is nuanced. We adopt a **transparent flagging** approach:

1. **Flag but don't delete by default** — Preserve all observations; mark suspicious ones
2. **Multiple QC passes** — Different analyses may require different thresholds
3. **Report impact** — Show how QC choices affect annual budgets

### QC Components

1. **Missingness analysis** — Identify gaps in the record
2. **Outlier detection** — Flag extreme values using Median Absolute Deviation (MAD)
3. **Aggregation stability** — Check if different aggregation methods yield consistent results


In [ ]:
# ============================================================================
# QC Functions
# ============================================================================

def compute_missingness(df: pd.DataFrame, variables: List[str] = None) -> Dict:
    """
    Compute missingness statistics for a site DataFrame.
    
    Returns:
    --------
    Dict with:
        - overall: percent missing per variable
        - by_month: missing percent by month
        - by_year: missing percent by year
    """
    if variables is None:
        variables = ['nee_raw', 'temp', 'radiation', 'vpd', 'swc']
    
    # Filter to existing variables
    variables = [v for v in variables if v in df.columns]
    
    result = {
        'overall': {},
        'by_month': {},
        'by_year': {}
    }
    
    for var in variables:
        # Overall missingness
        total = len(df)
        missing = df[var].isna().sum()
        result['overall'][var] = 100 * missing / total if total > 0 else 0
        
        # By month
        by_month = df.groupby('month')[var].apply(lambda x: 100 * x.isna().sum() / len(x))
        result['by_month'][var] = by_month.to_dict()
        
        # By year
        by_year = df.groupby('year')[var].apply(lambda x: 100 * x.isna().sum() / len(x))
        result['by_year'][var] = by_year.to_dict()
    
    return result

def flag_outliers_mad(series: pd.Series, threshold: float = 3.5) -> pd.Series:
    """
    Flag outliers using Median Absolute Deviation (MAD).
    
    MAD is more robust than standard deviation for non-normal distributions.
    threshold=3.5 corresponds to ~99.7% of a normal distribution.
    
    Returns:
    --------
    Boolean Series: True = outlier
    """
    # Remove NaN for calculation
    valid = series.dropna()
    
    if len(valid) == 0:
        return pd.Series(False, index=series.index)
    
    median = valid.median()
    mad = np.median(np.abs(valid - median))
    
    # Avoid division by zero
    if mad == 0:
        mad = valid.std() * 0.6745  # Convert to MAD scale
        if mad == 0:
            return pd.Series(False, index=series.index)
    
    # Modified Z-score
    modified_z = 0.6745 * (series - median) / mad
    
    return np.abs(modified_z) > threshold

def qc_flags(df: pd.DataFrame, nee_col: str = 'nee_raw', 
             outlier_threshold: float = 3.5) -> pd.DataFrame:
    """
    Apply QC flagging to a site DataFrame.
    
    Adds columns:
        - is_outlier: boolean, True if NEE is a statistical outlier
        - nee_qcd: NEE with outliers set to NaN
    
    Returns:
    --------
    DataFrame with QC columns added
    """
    df = df.copy()
    
    # Flag outliers
    df['is_outlier'] = flag_outliers_mad(df[nee_col], threshold=outlier_threshold)
    
    # Create QC'd NEE column (outliers removed)
    df['nee_qcd'] = df[nee_col].copy()
    df.loc[df['is_outlier'], 'nee_qcd'] = np.nan
    
    # Also flag based on existing QC flags if available
    if 'nee_qc' in df.columns:
        # FLUXNET QC: 0=measured, 1=good gap-fill, 2=medium, 3=poor
        df['is_poor_qc'] = df['nee_qc'] >= 3
    
    return df

def annual_budget(df: pd.DataFrame, nee_col: str = 'nee_raw', 
                  method: str = 'daily_sum') -> pd.DataFrame:
    """
    Calculate annual carbon budgets using different methods.
    
    Methods:
    - 'daily_sum': Sum daily NEE values
    - 'monthly_sum': Aggregate to monthly first, then sum
    - 'qcd_sum': Sum QC'd daily values (outliers excluded)
    
    Returns:
    --------
    DataFrame with columns: [year, method, annual_total, n_valid_days]
    
    Units: g C m⁻² yr⁻¹ (assuming input is g C m⁻² day⁻¹)
    """
    results = []
    
    for year in df['year'].unique():
        year_data = df[df['year'] == year]
        
        if method == 'daily_sum':
            total = year_data[nee_col].sum()
            n_valid = year_data[nee_col].notna().sum()
        
        elif method == 'monthly_sum':
            # Aggregate to monthly means first, then sum × 30.44 (avg days/month)
            monthly = year_data.groupby('month')[nee_col].mean()
            total = monthly.sum() * 30.44
            n_valid = year_data[nee_col].notna().sum()
        
        elif method == 'qcd_sum':
            if 'nee_qcd' in df.columns:
                total = year_data['nee_qcd'].sum()
                n_valid = year_data['nee_qcd'].notna().sum()
            else:
                total = np.nan
                n_valid = 0
        
        else:
            raise ValueError(f"Unknown method: {method}")
        
        results.append({
            'year': year,
            'method': method,
            'annual_total': total,
            'n_valid_days': n_valid
        })
    
    return pd.DataFrame(results)

print("✓ QC functions defined")


In [ ]:
# ============================================================================
# Apply QC to All Sites
# ============================================================================

print("Applying QC flagging to all sites...")
print("=" * 70)

qc_summary = []

for site_id in site_data:
    df = site_data[site_id]
    
    # Apply QC
    df_qc = qc_flags(df)
    site_data[site_id] = df_qc  # Update stored data
    
    # Calculate statistics
    n_total = len(df_qc)
    n_outliers = df_qc['is_outlier'].sum()
    outlier_pct = 100 * n_outliers / n_total
    
    # Missingness
    missing_stats = compute_missingness(df_qc)
    nee_missing = missing_stats['overall'].get('nee_raw', 0)
    
    qc_summary.append({
        'Site': site_id,
        'Total Days': n_total,
        'NEE Missing (%)': f"{nee_missing:.1f}",
        'Outliers Flagged': n_outliers,
        'Outlier (%)': f"{outlier_pct:.2f}"
    })
    
    print(f"  {site_id}: {n_outliers} outliers flagged ({outlier_pct:.2f}%)")

qc_summary_df = pd.DataFrame(qc_summary)
print("\n" + "=" * 70)
print("\n📊 QC SUMMARY")
print(qc_summary_df.to_string(index=False))

# Save QC summary
qc_summary_df.to_csv(OUTPUT_TABLES / 'qc_summary.csv', index=False)
print(f"\n✓ QC summary saved to {OUTPUT_TABLES / 'qc_summary.csv'}")


In [ ]:
# ============================================================================
# Aggregation Stability Check
# ============================================================================

print("Comparing annual totals across aggregation methods...")
print("=" * 70)

aggregation_results = []

for site_id in site_data:
    df = site_data[site_id]
    
    # Calculate annual budgets using different methods
    for method in ['daily_sum', 'monthly_sum', 'qcd_sum']:
        budget_df = annual_budget(df, method=method)
        budget_df['site_id'] = site_id
        aggregation_results.append(budget_df)

# Combine all results
all_budgets = pd.concat(aggregation_results, ignore_index=True)

# Pivot to compare methods
comparison = all_budgets.pivot_table(
    index=['site_id', 'year'], 
    columns='method', 
    values='annual_total'
).reset_index()

# Calculate differences between methods
if 'daily_sum' in comparison.columns and 'monthly_sum' in comparison.columns:
    comparison['daily_vs_monthly_diff'] = comparison['daily_sum'] - comparison['monthly_sum']
    comparison['diff_pct'] = 100 * comparison['daily_vs_monthly_diff'] / comparison['daily_sum'].abs()

print("\n📊 ANNUAL TOTALS BY METHOD (g C m⁻² yr⁻¹)")
print("=" * 100)
print(comparison.head(18).to_string(index=False))

# Summary statistics
print("\n📊 METHOD COMPARISON SUMMARY")
print("-" * 50)
if 'diff_pct' in comparison.columns:
    mean_diff = comparison['diff_pct'].abs().mean()
    max_diff = comparison['diff_pct'].abs().max()
    print(f"Mean |daily - monthly| difference: {mean_diff:.2f}%")
    print(f"Max  |daily - monthly| difference: {max_diff:.2f}%")
    
    if mean_diff < 5:
        print("→ Aggregation methods are reasonably consistent.")
    else:
        print("→ ⚠ Significant differences between methods detected. Investigate data gaps.")

# Save full comparison
comparison.to_csv(OUTPUT_TABLES / 'annual_budget_comparison.csv', index=False)
print(f"\n✓ Comparison saved to {OUTPUT_TABLES / 'annual_budget_comparison.csv'}")


---

## 4. Visualizations

### Publication-Style Plots

All plots use consistent styling and include:
- Clear axis labels with units
- Legends where appropriate
- Color coding: warm tones for Spain, cool tones for Ireland
- Saved to `outputs/figures/` in high resolution


In [ ]:
# ============================================================================
# Plot 1: Daily NEE Time Series (Raw vs QC'd)
# ============================================================================

def plot_nee_timeseries(site_data: Dict[str, pd.DataFrame], 
                        save_path: Path = None) -> None:
    """Plot daily NEE time series for all sites, showing raw vs QC'd data."""
    
    n_sites = len(site_data)
    fig, axes = plt.subplots(n_sites, 1, figsize=(14, 3 * n_sites), sharex=False)
    
    if n_sites == 1:
        axes = [axes]
    
    for ax, (site_id, df) in zip(axes, site_data.items()):
        color = ALL_SITE_COLORS[site_id]
        meta = SITE_METADATA[site_id]
        
        # Plot raw NEE
        ax.plot(df.index, df['nee_raw'], color=color, alpha=0.4, 
                linewidth=0.5, label='Raw NEE')
        
        # Overlay QC'd NEE
        if 'nee_qcd' in df.columns:
            ax.plot(df.index, df['nee_qcd'], color=color, alpha=0.9,
                    linewidth=0.8, label='QC\'d NEE')
        
        # Mark outliers
        outliers = df[df['is_outlier']]
        if len(outliers) > 0:
            ax.scatter(outliers.index, outliers['nee_raw'], 
                      color='red', s=10, alpha=0.7, marker='x',
                      label=f'Outliers (n={len(outliers)})', zorder=5)
        
        # Reference line at zero
        ax.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
        
        # Labels
        country = "🇪🇸" if site_id.startswith("ES") else "🇮🇪"
        ax.set_title(f"{country} {site_id}: {meta['name']} ({meta['igbp']})", 
                    fontsize=12, fontweight='bold')
        ax.set_ylabel('NEE\n(g C m⁻² day⁻¹)', fontsize=10)
        ax.legend(loc='upper right', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # Format x-axis
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    
    axes[-1].set_xlabel('Date', fontsize=11)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_nee_timeseries(site_data, OUTPUT_FIGURES / 'nee_timeseries_all_sites.png')


In [ ]:
# ============================================================================
# Plot 2: Seasonal Cycle (Monthly Climatology)
# ============================================================================

def plot_seasonal_cycle(site_data: Dict[str, pd.DataFrame],
                        save_path: Path = None) -> None:
    """Plot monthly climatology of NEE for all sites."""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    month_names = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
    
    # Left panel: Spanish sites
    ax1 = axes[0]
    for site_id in SPANISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        monthly = df.groupby('month')['nee_qcd'].agg(['mean', 'std'])
        
        color = ALL_SITE_COLORS[site_id]
        ax1.plot(monthly.index, monthly['mean'], 'o-', color=color, 
                linewidth=2, markersize=6, label=f"{site_id}")
        ax1.fill_between(monthly.index, 
                        monthly['mean'] - monthly['std'],
                        monthly['mean'] + monthly['std'],
                        color=color, alpha=0.2)
    
    ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax1.set_xlabel('Month', fontsize=11)
    ax1.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax1.set_title('🇪🇸 Spanish Sites (Mediterranean)', fontsize=12, fontweight='bold')
    ax1.set_xticks(range(1, 13))
    ax1.set_xticklabels(month_names)
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Right panel: Irish sites
    ax2 = axes[1]
    for site_id in IRISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        monthly = df.groupby('month')['nee_qcd'].agg(['mean', 'std'])
        
        color = ALL_SITE_COLORS[site_id]
        ax2.plot(monthly.index, monthly['mean'], 'o-', color=color,
                linewidth=2, markersize=6, label=f"{site_id}")
        ax2.fill_between(monthly.index,
                        monthly['mean'] - monthly['std'],
                        monthly['mean'] + monthly['std'],
                        color=color, alpha=0.2)
    
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax2.set_xlabel('Month', fontsize=11)
    ax2.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax2.set_title('🇮🇪 Irish Sites (Atlantic)', fontsize=12, fontweight='bold')
    ax2.set_xticks(range(1, 13))
    ax2.set_xticklabels(month_names)
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    # Add interpretation note
    fig.text(0.5, -0.02, 
            'Shaded areas: ±1 standard deviation across years. Negative = net uptake, Positive = net release.',
            ha='center', fontsize=10, style='italic', color='#555555')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_seasonal_cycle(site_data, OUTPUT_FIGURES / 'seasonal_cycle_spain_ireland.png')


In [ ]:
# ============================================================================
# Plot 3: Missingness Heatmap
# ============================================================================

def plot_missingness_heatmap(site_data: Dict[str, pd.DataFrame],
                             save_path: Path = None) -> None:
    """Create heatmap showing data availability by month/year for each site."""
    
    n_sites = len(site_data)
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()
    
    for ax, (site_id, df) in zip(axes, site_data.items()):
        # Create year-month pivot table of valid data percentage
        df_temp = df.copy()
        df_temp['valid'] = df_temp['nee_raw'].notna().astype(int)
        
        # Group by year and month
        pivot = df_temp.pivot_table(
            index='year', 
            columns='month', 
            values='valid',
            aggfunc='mean'
        ) * 100  # Convert to percentage
        
        # Create heatmap
        sns.heatmap(pivot, ax=ax, cmap='RdYlGn', vmin=0, vmax=100,
                   annot=False, cbar_kws={'label': '% Valid'}, 
                   linewidths=0.5, linecolor='white')
        
        # Labels
        country = "🇪🇸" if site_id.startswith("ES") else "🇮🇪"
        ax.set_title(f"{country} {site_id}", fontsize=11, fontweight='bold')
        ax.set_xlabel('Month')
        ax.set_ylabel('Year')
        ax.set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'], 
                          rotation=0, fontsize=9)
    
    # Hide unused axes if any
    for ax in axes[len(site_data):]:
        ax.set_visible(False)
    
    plt.suptitle('NEE Data Availability by Month and Year\n(Green = complete, Red = missing)',
                fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_missingness_heatmap(site_data, OUTPUT_FIGURES / 'missingness_heatmap.png')


In [ ]:
# ============================================================================
# Plot 4: NEE vs Environmental Drivers (Temperature, Radiation)
# ============================================================================

def plot_nee_vs_drivers(site_data: Dict[str, pd.DataFrame],
                        save_path: Path = None) -> None:
    """Scatter plots of NEE vs temperature and radiation."""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Top row: NEE vs Temperature
    # Left: Spanish sites
    ax1 = axes[0, 0]
    for site_id in SPANISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[['nee_qcd', 'temp']].dropna()
        ax1.scatter(valid['temp'], valid['nee_qcd'], 
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=10, label=site_id)
    ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax1.set_xlabel('Air Temperature (°C)', fontsize=11)
    ax1.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax1.set_title('🇪🇸 Spain: NEE vs Temperature', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper right', markerscale=2)
    ax1.grid(True, alpha=0.3)
    
    # Right: Irish sites
    ax2 = axes[0, 1]
    for site_id in IRISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[['nee_qcd', 'temp']].dropna()
        ax2.scatter(valid['temp'], valid['nee_qcd'],
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=10, label=site_id)
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax2.set_xlabel('Air Temperature (°C)', fontsize=11)
    ax2.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax2.set_title('🇮🇪 Ireland: NEE vs Temperature', fontsize=12, fontweight='bold')
    ax2.legend(loc='upper right', markerscale=2)
    ax2.grid(True, alpha=0.3)
    
    # Bottom row: NEE vs Radiation
    # Left: Spanish sites
    ax3 = axes[1, 0]
    for site_id in SPANISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[['nee_qcd', 'radiation']].dropna()
        ax3.scatter(valid['radiation'], valid['nee_qcd'],
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=10, label=site_id)
    ax3.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax3.set_xlabel('Incoming Shortwave Radiation (W m⁻²)', fontsize=11)
    ax3.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax3.set_title('🇪🇸 Spain: NEE vs Radiation', fontsize=12, fontweight='bold')
    ax3.legend(loc='upper right', markerscale=2)
    ax3.grid(True, alpha=0.3)
    
    # Right: Irish sites
    ax4 = axes[1, 1]
    for site_id in IRISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[['nee_qcd', 'radiation']].dropna()
        ax4.scatter(valid['radiation'], valid['nee_qcd'],
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=10, label=site_id)
    ax4.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax4.set_xlabel('Incoming Shortwave Radiation (W m⁻²)', fontsize=11)
    ax4.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax4.set_title('🇮🇪 Ireland: NEE vs Radiation', fontsize=12, fontweight='bold')
    ax4.legend(loc='upper right', markerscale=2)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_nee_vs_drivers(site_data, OUTPUT_FIGURES / 'nee_vs_drivers.png')


---

## 5. Cumulative Carbon Budget Analysis

### Converting NEE to Carbon Budget

The cumulative sum of daily NEE gives the total carbon exchange over time:
- **Negative cumulative** → net carbon sink (ecosystem absorbed CO₂)
- **Positive cumulative** → net carbon source (ecosystem released CO₂)

We compute:
1. **Cumulative curves** — Running sum over each year
2. **Annual totals** — Final value at end of year
3. **Method comparison** — Raw vs QC'd vs monthly-aggregated approaches


In [ ]:
# ============================================================================
# Cumulative Carbon Budget Calculation
# ============================================================================

def compute_cumulative_nee(df: pd.DataFrame, nee_col: str = 'nee_qcd') -> pd.DataFrame:
    """
    Compute cumulative NEE for each year.
    
    Returns DataFrame with 'cumsum' column added.
    """
    df = df.copy()
    
    # Compute cumulative sum by year
    df['cumsum'] = df.groupby('year')[nee_col].cumsum()
    
    return df

def plot_cumulative_budget(site_data: Dict[str, pd.DataFrame],
                           years_to_show: int = 3,
                           save_path: Path = None) -> None:
    """Plot cumulative NEE curves for selected years."""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Get common years across sites
    all_years = set()
    for df in site_data.values():
        all_years.update(df['year'].unique())
    common_years = sorted(list(all_years))[-years_to_show:]  # Last N years
    
    linestyles = ['-', '--', ':']
    
    # Left: Spanish sites
    ax1 = axes[0]
    for site_id in SPANISH_SITES:
        if site_id not in site_data:
            continue
        df = compute_cumulative_nee(site_data[site_id])
        color = ALL_SITE_COLORS[site_id]
        
        for i, year in enumerate(common_years):
            year_data = df[df['year'] == year]
            if len(year_data) == 0:
                continue
            ax1.plot(year_data['doy'], year_data['cumsum'],
                    color=color, linestyle=linestyles[i % len(linestyles)],
                    linewidth=1.5, alpha=0.8,
                    label=f"{site_id} ({year})")
    
    ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax1.set_xlabel('Day of Year', fontsize=11)
    ax1.set_ylabel('Cumulative NEE (g C m⁻²)', fontsize=11)
    ax1.set_title('🇪🇸 Spanish Sites: Cumulative Carbon Budget', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper left', fontsize=8, ncol=2)
    ax1.grid(True, alpha=0.3)
    
    # Right: Irish sites
    ax2 = axes[1]
    for site_id in IRISH_SITES:
        if site_id not in site_data:
            continue
        df = compute_cumulative_nee(site_data[site_id])
        color = ALL_SITE_COLORS[site_id]
        
        for i, year in enumerate(common_years):
            year_data = df[df['year'] == year]
            if len(year_data) == 0:
                continue
            ax2.plot(year_data['doy'], year_data['cumsum'],
                    color=color, linestyle=linestyles[i % len(linestyles)],
                    linewidth=1.5, alpha=0.8,
                    label=f"{site_id} ({year})")
    
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax2.set_xlabel('Day of Year', fontsize=11)
    ax2.set_ylabel('Cumulative NEE (g C m⁻²)', fontsize=11)
    ax2.set_title('🇮🇪 Irish Sites: Cumulative Carbon Budget', fontsize=12, fontweight='bold')
    ax2.legend(loc='upper left', fontsize=8, ncol=2)
    ax2.grid(True, alpha=0.3)
    
    # Add interpretation note
    fig.text(0.5, -0.02,
            'Below zero = net carbon sink (uptake). Above zero = net carbon source (release).',
            ha='center', fontsize=10, style='italic', color='#555555')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_cumulative_budget(site_data, years_to_show=3, 
                       save_path=OUTPUT_FIGURES / 'cumulative_budget.png')


In [ ]:
# ============================================================================
# Annual Totals Summary Table
# ============================================================================

def create_annual_totals_table(site_data: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    Create a tidy table of annual carbon totals for all sites and methods.
    
    Format: [site, year, method, annual_total]
    """
    all_rows = []
    
    for site_id, df in site_data.items():
        for method in ['daily_sum', 'qcd_sum', 'monthly_sum']:
            budget_df = annual_budget(df, method=method)
            for _, row in budget_df.iterrows():
                all_rows.append({
                    'site': site_id,
                    'country': 'Spain' if site_id.startswith('ES') else 'Ireland',
                    'year': int(row['year']),
                    'method': method,
                    'annual_total_gCm2': round(row['annual_total'], 2),
                    'n_valid_days': int(row['n_valid_days'])
                })
    
    return pd.DataFrame(all_rows)

# Create and display table
annual_table = create_annual_totals_table(site_data)

# Show pivot summary
pivot = annual_table.pivot_table(
    index=['country', 'site', 'year'],
    columns='method',
    values='annual_total_gCm2'
).round(1)

print("\n📊 ANNUAL CARBON TOTALS (g C m⁻² yr⁻¹)")
print("=" * 80)
print(pivot.head(20).to_string())
print("\n(Negative = net sink, Positive = net source)")

# Save full table
annual_table.to_csv(OUTPUT_TABLES / 'annual_carbon_totals.csv', index=False)
print(f"\n✓ Full table saved to {OUTPUT_TABLES / 'annual_carbon_totals.csv'}")


---

## 6. Uncertainty Quantification via Bootstrap

### Bootstrap Approach

To quantify uncertainty in annual carbon totals, we use **bootstrap resampling**:

1. For each site-year, resample daily NEE values with replacement (1000 iterations)
2. Compute the annual total for each bootstrap sample
3. Report the distribution (mean, 2.5th–97.5th percentiles)

### Important Caveat

**This captures sampling uncertainty** given the observed day-to-day variability. It does NOT capture:
- Systematic measurement errors in the eddy covariance system
- Errors introduced by gap-filling algorithms
- Representativeness errors (footprint heterogeneity)
- Calibration uncertainties

For a complete uncertainty budget, additional methods would be needed.


In [ ]:
# ============================================================================
# Bootstrap Uncertainty Analysis
# ============================================================================

def bootstrap_annual_totals(series: pd.Series, n_bootstrap: int = 1000,
                           random_state: int = 42) -> np.ndarray:
    """
    Bootstrap resample daily NEE to estimate uncertainty in annual total.
    
    Parameters:
    -----------
    series : pd.Series
        Daily NEE values (can contain NaN)
    n_bootstrap : int
        Number of bootstrap iterations
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    np.ndarray of bootstrap annual totals
    """
    # Drop NaN values
    valid = series.dropna().values
    n = len(valid)
    
    if n == 0:
        return np.full(n_bootstrap, np.nan)
    
    np.random.seed(random_state)
    
    # Bootstrap: resample with replacement, sum to get annual total
    bootstrap_totals = np.zeros(n_bootstrap)
    
    for i in range(n_bootstrap):
        # Resample n days with replacement
        sample = np.random.choice(valid, size=n, replace=True)
        bootstrap_totals[i] = np.sum(sample)
    
    return bootstrap_totals

def compute_all_bootstrap_estimates(site_data: Dict[str, pd.DataFrame],
                                   n_bootstrap: int = 1000) -> pd.DataFrame:
    """
    Compute bootstrap uncertainty estimates for all site-years.
    """
    results = []
    
    for site_id, df in site_data.items():
        for year in df['year'].unique():
            year_data = df[df['year'] == year]['nee_qcd']
            
            # Bootstrap
            bootstrap_dist = bootstrap_annual_totals(year_data, n_bootstrap)
            
            # Compute statistics
            mean_total = np.nanmean(bootstrap_dist)
            std_total = np.nanstd(bootstrap_dist)
            ci_lower = np.nanpercentile(bootstrap_dist, 2.5)
            ci_upper = np.nanpercentile(bootstrap_dist, 97.5)
            
            results.append({
                'site': site_id,
                'country': 'Spain' if site_id.startswith('ES') else 'Ireland',
                'year': int(year),
                'mean': round(mean_total, 2),
                'std': round(std_total, 2),
                'ci_2.5%': round(ci_lower, 2),
                'ci_97.5%': round(ci_upper, 2),
                'ci_width': round(ci_upper - ci_lower, 2)
            })
    
    return pd.DataFrame(results)

# Run bootstrap analysis
print("Running bootstrap analysis (1000 iterations per site-year)...")
print("This may take a moment...")

bootstrap_results = compute_all_bootstrap_estimates(site_data, n_bootstrap=1000)

print("\n📊 BOOTSTRAP UNCERTAINTY ESTIMATES (g C m⁻² yr⁻¹)")
print("=" * 90)
print(bootstrap_results.to_string(index=False))

# Save results
bootstrap_results.to_csv(OUTPUT_TABLES / 'bootstrap_uncertainty.csv', index=False)
print(f"\n✓ Bootstrap results saved to {OUTPUT_TABLES / 'bootstrap_uncertainty.csv'}")


In [ ]:
# ============================================================================
# Bootstrap Visualization
# ============================================================================

def plot_bootstrap_distributions(site_data: Dict[str, pd.DataFrame],
                                 year: int = None,
                                 save_path: Path = None) -> None:
    """
    Plot violin plots of bootstrap distributions for each site.
    """
    if year is None:
        # Find a year with data from most sites
        all_years = set()
        for df in site_data.values():
            all_years.update(df['year'].unique())
        year = max(all_years)  # Use most recent year
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Collect bootstrap distributions
    spain_data = {}
    ireland_data = {}
    
    for site_id, df in site_data.items():
        year_data = df[df['year'] == year]['nee_qcd']
        if len(year_data) == 0:
            continue
        
        bootstrap_dist = bootstrap_annual_totals(year_data, n_bootstrap=1000)
        
        if site_id.startswith('ES'):
            spain_data[site_id] = bootstrap_dist
        else:
            ireland_data[site_id] = bootstrap_dist
    
    # Left panel: Spain violin plot
    ax1 = axes[0]
    if spain_data:
        positions = range(len(spain_data))
        data_list = list(spain_data.values())
        labels = list(spain_data.keys())
        colors = [ALL_SITE_COLORS[s] for s in labels]
        
        parts = ax1.violinplot(data_list, positions=positions, showmeans=True, showmedians=True)
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors[i])
            pc.set_alpha(0.7)
        
        ax1.set_xticks(positions)
        ax1.set_xticklabels(labels, fontsize=11)
        ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    
    ax1.set_ylabel('Annual NEE (g C m⁻² yr⁻¹)', fontsize=11)
    ax1.set_title(f'🇪🇸 Spanish Sites: Bootstrap Distribution ({year})', 
                 fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Right panel: Ireland violin plot
    ax2 = axes[1]
    if ireland_data:
        positions = range(len(ireland_data))
        data_list = list(ireland_data.values())
        labels = list(ireland_data.keys())
        colors = [ALL_SITE_COLORS[s] for s in labels]
        
        parts = ax2.violinplot(data_list, positions=positions, showmeans=True, showmedians=True)
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(colors[i])
            pc.set_alpha(0.7)
        
        ax2.set_xticks(positions)
        ax2.set_xticklabels(labels, fontsize=11)
        ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    
    ax2.set_ylabel('Annual NEE (g C m⁻² yr⁻¹)', fontsize=11)
    ax2.set_title(f'🇮🇪 Irish Sites: Bootstrap Distribution ({year})',
                 fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add interpretation
    fig.text(0.5, -0.02,
            'Violin shows bootstrap distribution (n=1000). White dot = median, black bar = mean.',
            ha='center', fontsize=10, style='italic', color='#555555')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_bootstrap_distributions(site_data, save_path=OUTPUT_FIGURES / 'bootstrap_violins.png')


---

## 7. Water Stress Signal: Spain vs Ireland

### The VPD-Driven Hypothesis

In Mediterranean climates, high **Vapor Pressure Deficit (VPD)** during summer forces stomatal closure, reducing photosynthetic CO₂ uptake even when radiation is abundant. This creates a characteristic "summer uptake collapse".

In Atlantic Ireland, VPD remains relatively low year-round, allowing more consistent growing-season uptake.

### Analysis Approach

1. **VPD binning**: Classify days into low/medium/high VPD categories
2. **Conditional NEE**: Compare mean NEE across VPD bins
3. **Seasonal × VPD interaction**: How does this vary by month?

If VPD data are unavailable, we note this limitation and fall back to temperature as a proxy (imperfect, but correlated with evaporative demand).


In [ ]:
# ============================================================================
# Water Stress Analysis
# ============================================================================

def check_vpd_availability(site_data: Dict[str, pd.DataFrame]) -> Dict[str, float]:
    """Check VPD data availability across sites."""
    availability = {}
    for site_id, df in site_data.items():
        if 'vpd' in df.columns:
            valid_pct = 100 * df['vpd'].notna().sum() / len(df)
            availability[site_id] = valid_pct
        else:
            availability[site_id] = 0.0
    return availability

# Check VPD availability
vpd_avail = check_vpd_availability(site_data)
print("📊 VPD DATA AVAILABILITY")
print("-" * 40)
for site_id, pct in vpd_avail.items():
    status = "✓" if pct > 50 else "⚠" if pct > 0 else "✗"
    print(f"  {status} {site_id}: {pct:.1f}% valid")

# Determine stress variable
use_vpd = all(pct > 50 for pct in vpd_avail.values())
stress_var = 'vpd' if use_vpd else 'temp'
stress_label = 'VPD (hPa)' if use_vpd else 'Temperature (°C, as proxy)'

if not use_vpd:
    print(f"\n⚠ VPD data incomplete. Using temperature as water stress proxy.")
    print("  Note: Temperature is an imperfect proxy for atmospheric dryness.")


In [ ]:
# ============================================================================
# VPD Binning and Conditional NEE Analysis
# ============================================================================

def compute_stress_bins(site_data: Dict[str, pd.DataFrame], 
                       stress_var: str = 'vpd',
                       n_bins: int = 3) -> pd.DataFrame:
    """
    Bin data by stress variable and compute mean NEE per bin.
    """
    all_stress = []
    for df in site_data.values():
        if stress_var in df.columns:
            all_stress.extend(df[stress_var].dropna().values)
    
    # Create bin edges based on tertiles of all data
    bin_edges = np.percentile(all_stress, [0, 33, 67, 100])
    bin_labels = ['Low', 'Medium', 'High']
    
    results = []
    
    for site_id, df in site_data.items():
        if stress_var not in df.columns:
            continue
        
        df_temp = df.copy()
        df_temp['stress_bin'] = pd.cut(df_temp[stress_var], bins=bin_edges, 
                                       labels=bin_labels, include_lowest=True)
        
        for bin_label in bin_labels:
            bin_data = df_temp[df_temp['stress_bin'] == bin_label]['nee_qcd']
            
            results.append({
                'site': site_id,
                'country': 'Spain' if site_id.startswith('ES') else 'Ireland',
                'stress_bin': bin_label,
                'mean_nee': bin_data.mean(),
                'std_nee': bin_data.std(),
                'n_days': len(bin_data.dropna())
            })
    
    return pd.DataFrame(results), bin_edges

# Compute stress bin analysis
stress_df, bin_edges = compute_stress_bins(site_data, stress_var=stress_var)

print(f"\n📊 NEE BY {stress_var.upper()} BINS")
print(f"   Bin edges: Low < {bin_edges[1]:.1f}, Medium < {bin_edges[2]:.1f}, High > {bin_edges[2]:.1f}")
print("=" * 70)

# Pivot for display
pivot = stress_df.pivot_table(
    index=['country', 'site'],
    columns='stress_bin',
    values='mean_nee'
)[['Low', 'Medium', 'High']]

print(pivot.round(2).to_string())

# Key insight
spain_high = stress_df[(stress_df['country'] == 'Spain') & 
                       (stress_df['stress_bin'] == 'High')]['mean_nee'].mean()
ireland_high = stress_df[(stress_df['country'] == 'Ireland') & 
                         (stress_df['stress_bin'] == 'High')]['mean_nee'].mean()

print("\n🔍 KEY FINDING:")
print(f"   Mean NEE at high {stress_var}:")
print(f"   - Spain:   {spain_high:.2f} g C m⁻² day⁻¹")
print(f"   - Ireland: {ireland_high:.2f} g C m⁻² day⁻¹")
if spain_high > ireland_high:
    print(f"   → Spanish sites show reduced uptake (or source Behaviour) at high {stress_var}.")


In [ ]:
# ============================================================================
# Water Stress Visualization: NEE vs VPD/Temperature
# ============================================================================

def plot_nee_vs_stress(site_data: Dict[str, pd.DataFrame],
                       stress_var: str = 'vpd',
                       save_path: Path = None) -> None:
    """
    Plot NEE vs stress variable (VPD or temperature) comparing Spain vs Ireland.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    stress_label = 'VPD (hPa)' if stress_var == 'vpd' else 'Temperature (°C)'
    
    # Left: Spain
    ax1 = axes[0]
    for site_id in SPANISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[[stress_var, 'nee_qcd']].dropna()
        
        ax1.scatter(valid[stress_var], valid['nee_qcd'],
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=8, label=site_id)
    
    # Add trend line for Spain
    spain_stress = []
    spain_nee = []
    for site_id in SPANISH_SITES:
        if site_id in site_data:
            df = site_data[site_id]
            valid = df[[stress_var, 'nee_qcd']].dropna()
            spain_stress.extend(valid[stress_var].values)
            spain_nee.extend(valid['nee_qcd'].values)
    
    if SCIPY_AVAILABLE and len(spain_stress) > 10:
        slope, intercept, r, p, se = stats.linregress(spain_stress, spain_nee)
        x_line = np.linspace(min(spain_stress), max(spain_stress), 100)
        y_line = slope * x_line + intercept
        ax1.plot(x_line, y_line, 'k--', linewidth=2, 
                label=f'Trend (r={r:.2f})')
    
    ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax1.set_xlabel(stress_label, fontsize=11)
    ax1.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax1.set_title('🇪🇸 Spanish Sites', fontsize=12, fontweight='bold')
    ax1.legend(loc='upper right', markerscale=2, fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Right: Ireland
    ax2 = axes[1]
    for site_id in IRISH_SITES:
        if site_id not in site_data:
            continue
        df = site_data[site_id]
        valid = df[[stress_var, 'nee_qcd']].dropna()
        
        ax2.scatter(valid[stress_var], valid['nee_qcd'],
                   c=ALL_SITE_COLORS[site_id], alpha=0.3, s=8, label=site_id)
    
    # Add trend line for Ireland
    ireland_stress = []
    ireland_nee = []
    for site_id in IRISH_SITES:
        if site_id in site_data:
            df = site_data[site_id]
            valid = df[[stress_var, 'nee_qcd']].dropna()
            ireland_stress.extend(valid[stress_var].values)
            ireland_nee.extend(valid['nee_qcd'].values)
    
    if SCIPY_AVAILABLE and len(ireland_stress) > 10:
        slope, intercept, r, p, se = stats.linregress(ireland_stress, ireland_nee)
        x_line = np.linspace(min(ireland_stress), max(ireland_stress), 100)
        y_line = slope * x_line + intercept
        ax2.plot(x_line, y_line, 'k--', linewidth=2,
                label=f'Trend (r={r:.2f})')
    
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
    ax2.set_xlabel(stress_label, fontsize=11)
    ax2.set_ylabel('NEE (g C m⁻² day⁻¹)', fontsize=11)
    ax2.set_title('🇮🇪 Irish Sites', fontsize=12, fontweight='bold')
    ax2.legend(loc='upper right', markerscale=2, fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    # Match y-axes
    y_min = min(ax1.get_ylim()[0], ax2.get_ylim()[0])
    y_max = max(ax1.get_ylim()[1], ax2.get_ylim()[1])
    ax1.set_ylim(y_min, y_max)
    ax2.set_ylim(y_min, y_max)
    
    plt.suptitle(f'NEE Response to {stress_label.split(" ")[0]}: Spain vs Ireland',
                fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
        print(f"✓ Saved: {save_path}")
    
    plt.show()

# Generate plot
plot_nee_vs_stress(site_data, stress_var=stress_var,
                   save_path=OUTPUT_FIGURES / 'nee_vs_water_stress.png')


---

## 8. Site Map and Metadata

### Geographic Context

The 6 sites span from southern Spain (~37°N) to central Ireland (~53°N), representing a strong climate gradient from Mediterranean to Atlantic regimes.


In [ ]:
# ============================================================================
# Site Map (using Folium if available)
# ============================================================================

def create_site_map(site_metadata: Dict, save_path: Path = None):
    """
    Create an interactive map of all sites using Folium.
    Falls back to matplotlib if Folium not available.
    """
    if FOLIUM_AVAILABLE:
        # Center map between Spain and Ireland
        center_lat = 45.0
        center_lon = -5.0
        
        m = folium.Map(location=[center_lat, center_lon], zoom_start=5,
                      tiles='CartoDB positron')
        
        for site_id, meta in site_metadata.items():
            # Color and icon based on country
            if site_id.startswith('ES'):
                color = 'red'
                icon = 'sun'
            else:
                color = 'green'
                icon = 'leaf'
            
            # Create popup content
            popup_html = f"""
            <b>{site_id}</b>: {meta['name']}<br>
            <b>Country:</b> {meta['country']}<br>
            <b>IGBP:</b> {meta['igbp_full']}<br>
            <b>Climate:</b> {meta['climate']}<br>
            <b>Lat/Lon:</b> {meta['lat']:.4f}, {meta['lon']:.4f}<br>
            <i>{meta['notes']}</i>
            """
            
            folium.Marker(
                location=[meta['lat'], meta['lon']],
                popup=folium.Popup(popup_html, max_width=300),
                tooltip=f"{site_id}: {meta['name']}",
                icon=folium.Icon(color=color, icon=icon, prefix='fa')
            ).add_to(m)
        
        # Add legend
        legend_html = '''
        <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                    background-color: white; padding: 10px; border: 2px solid grey;
                    border-radius: 5px; font-size: 12px;">
            <b>FLUXNET Sites</b><br>
            <i class="fa fa-map-marker" style="color:red"></i> Spanish (Mediterranean)<br>
            <i class="fa fa-map-marker" style="color:green"></i> Irish (Atlantic)
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        if save_path:
            m.save(str(save_path))
            print(f"✓ Interactive map saved to {save_path}")
        
        return m
    
    else:
        # Fallback: matplotlib static map
        fig, ax = plt.subplots(figsize=(10, 8))
        
        for site_id, meta in site_metadata.items():
            color = ALL_SITE_COLORS[site_id]
            marker = 'o' if site_id.startswith('ES') else 's'
            ax.scatter(meta['lon'], meta['lat'], c=color, s=150, marker=marker,
                      edgecolors='black', linewidth=1.5, zorder=5)
            ax.annotate(site_id, (meta['lon'], meta['lat']), 
                       xytext=(5, 5), textcoords='offset points',
                       fontsize=10, fontweight='bold')
        
        ax.set_xlabel('Longitude', fontsize=11)
        ax.set_ylabel('Latitude', fontsize=11)
        ax.set_title('FLUXNET Site Locations: Spain vs Ireland', 
                    fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # Add country labels
        ax.text(-4, 38, 'SPAIN', fontsize=14, fontweight='bold', 
               color='#E74C3C', alpha=0.7)
        ax.text(-8, 53, 'IRELAND', fontsize=14, fontweight='bold',
               color='#27AE60', alpha=0.7)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
            print(f"✓ Static map saved to {save_path}")
        
        plt.show()
        return None

# Create map
if FOLIUM_AVAILABLE:
    site_map = create_site_map(SITE_METADATA, OUTPUT_FIGURES / 'site_map.html')
    display(site_map)
else:
    create_site_map(SITE_METADATA, OUTPUT_FIGURES / 'site_map.png')


In [ ]:
# ============================================================================
# Site Metadata Table
# ============================================================================

def create_metadata_table(site_metadata: Dict) -> pd.DataFrame:
    """Create a formatted metadata table for all sites."""
    rows = []
    
    for site_id, meta in site_metadata.items():
        rows.append({
            'Site ID': site_id,
            'Name': meta['name'],
            'Country': meta['country'],
            'Latitude': meta['lat'],
            'Longitude': meta['lon'],
            'IGBP Class': meta['igbp'],
            'Land Cover': meta['igbp_full'],
            'Climate': meta['climate'],
            'Notes': meta['notes']
        })
    
    return pd.DataFrame(rows)

# Create and display table
metadata_df = create_metadata_table(SITE_METADATA)

print("\n📊 SITE METADATA")
print("=" * 100)
display_cols = ['Site ID', 'Name', 'Country', 'Latitude', 'Longitude', 'IGBP Class', 'Climate']
print(metadata_df[display_cols].to_string(index=False))

# Save full metadata
metadata_df.to_csv(OUTPUT_TABLES / 'site_metadata.csv', index=False)
print(f"\n✓ Full metadata saved to {OUTPUT_TABLES / 'site_metadata.csv'}")

# Display notes
print("\n📝 SITE CONTEXT NOTES")
print("-" * 50)
for _, row in metadata_df.iterrows():
    country = "🇪🇸" if row['Country'] == 'Spain' else "🇮🇪"
    print(f"{country} {row['Site ID']}: {row['Notes']}")


---

## 9. Implications for Process Model Validation

This exploratory analysis reveals several patterns that would be relevant for validating ecosystem carbon models against eddy-covariance observations.

### Patterns a Process Model Would Need to Reproduce

1. **Seasonal Phase Differences**
   - Spanish sites: Peak uptake in spring (March-May) before summer drought
   - Irish sites: Peak uptake later in growing season (May-July)
   - A model must capture these phenological differences

2. **Summer "Uptake Collapse" in Mediterranean Systems**
   - Strong reduction or reversal of NEE during high-VPD summer months in Spain
   - Models need realistic stomatal conductance response to VPD
   - Water limitation must be explicitly represented

3. **Inter-Annual Variability**
   - Bootstrap analysis shows substantial year-to-year spread
   - Single-year validation is insufficient; multi-year testing required

4. **Diurnal to Annual Scale Consistency**
   - Daily aggregation methods show ~X% differences (from our stability check)
   - Model outputs at different scales must be internally consistent

### Aspects Most Sensitive to QC / Aggregation Choice

| Aspect | Sensitivity | Implication |
|--------|-------------|-------------|
| Annual totals | Medium-High | Outlier treatment changes budget by ~5-15% |
| Seasonal pattern | Low | Robust to QC approach |
| VPD response | Medium | Depends on which high-VPD days are flagged |
| Cross-site ranking | Low | Relative differences preserved |

### Why Cross-Site Comparison is Non-Trivial

1. **Different vegetation types**: Comparing shrubland (ES-LgS) to grassland (IE-Dri) mixes land cover and climate effects

2. **Different data lengths**: Sites have varying record lengths and coverage; direct comparison requires harmonization

3. **Different QC needs**: Mediterranean sites may have more extreme values that are legitimate (not errors)

4. **Gap-filling artifacts**: FLUXNET gap-filling methods may behave differently in dry vs. wet climates

### Recommendations for Model Validation

1. **Start with seasonal cycle reproduction** — easier target than absolute magnitudes

2. **Test VPD/drought response explicitly** — key differentiator between Spain and Ireland

3. **Report uncertainty ranges** — our bootstrap analysis provides a baseline for expected spread

4. **Use multiple years** — single-year agreement may be coincidental

5. **Be explicit about aggregation** — document daily vs. monthly summation approach


In [ ]:
# ============================================================================
# Final Summary Statistics
# ============================================================================

print("=" * 70)
print("📊 ANALYSIS SUMMARY")
print("=" * 70)

print(f"\n🌍 Sites Analyzed: {len(site_data)}")
print(f"   Spain: {len([s for s in site_data if s.startswith('ES')])}")
print(f"   Ireland: {len([s for s in site_data if s.startswith('IE')])}")

# Total data points
total_days = sum(len(df) for df in site_data.values())
print(f"\n📅 Total site-days: {total_days:,}")

# Annual budget summary
print(f"\n💰 Annual Carbon Budgets (g C m⁻² yr⁻¹):")
for country in ['Spain', 'Ireland']:
    sites = SPANISH_SITES if country == 'Spain' else IRISH_SITES
    country_means = []
    for site_id in sites:
        if site_id in site_data:
            df = site_data[site_id]
            annual = df.groupby('year')['nee_qcd'].sum().mean()
            country_means.append(annual)
    if country_means:
        mean_budget = np.mean(country_means)
        flag = "🇪🇸" if country == "Spain" else "🇮🇪"
        sink_source = "SINK" if mean_budget < 0 else "SOURCE"
        print(f"   {flag} {country}: {mean_budget:.1f} (net {sink_source})")

# Outputs generated
print(f"\n📁 Outputs Generated:")
print(f"   Figures: {OUTPUT_FIGURES}/")
for f in OUTPUT_FIGURES.glob('*.png'):
    print(f"      - {f.name}")
for f in OUTPUT_FIGURES.glob('*.html'):
    print(f"      - {f.name}")

print(f"\n   Tables: {OUTPUT_TABLES}/")
for f in OUTPUT_TABLES.glob('*.csv'):
    print(f"      - {f.name}")

print("\n" + "=" * 70)
print("✅ Analysis complete!")
print("=" * 70)


---

## Conclusion

This notebook demonstrates a systematic approach to exploratory analysis of eddy-covariance CO₂ flux data, comparing Mediterranean (Spain) and Atlantic (Ireland) climate regimes.

### Key Takeaways

1. **Data quality matters**: Simple QC flagging (MAD-based outliers) can change annual budgets by a meaningful margin. Transparency about QC choices is essential.

2. **Climate regimes drive phenology**: The seasonal pattern of carbon exchange differs fundamentally between drought-limited and water-abundant ecosystems.

3. **Uncertainty is real**: Bootstrap analysis reveals that even with complete data, day-to-day variability creates substantial uncertainty in annual totals.

4. **Cross-site comparison requires care**: Vegetation type, data coverage, and QC approaches all complicate direct comparisons.

### For the Teagasc Context

This analysis provides a foundation for understanding the measurement challenges that would be encountered when validating process models (e.g., for soil carbon predictions) against flux tower observations in Ireland. The contrast with Spanish sites highlights how Irish grasslands and croplands may exhibit:

- More consistent growing-season uptake
- Less extreme VPD-driven stomatal limitation
- Different sensitivities to temperature vs. moisture drivers

Understanding these measurement Behaviours is a prerequisite to meaningful model-data comparison.

---

*Analysis prepared December 2024*  
*Author: Robert (currently based near Gibraltar, Spain)*  
*Purpose: Teagasc Research Officer application - demonstration of measurement analysis skills*
